# **Data Analysis of Filming Permits of the city of Chicago**
## Although this dataset contains only a few features worth analyzing, it does not negatively impact the quality of the analysis.

In [ ]:
!pip install basemap basemap-data-hires

In [ ]:
#Libraries
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Functions
def get_variable_types(dataframe):
    continuous_vars = []
    categorical_vars = []

    for column in dataframe.columns:
        if dataframe[column].dtype == 'object':
            categorical_vars.append(column)
        else:
            continuous_vars.append(column)

    return continuous_vars, categorical_vars

# Plot the distribuition of a column
def plot_distribution(df, column):
    # Calculate value counts
    value_counts = df[column].value_counts()

    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Bar plot on the first subplot
    sns.barplot(x=value_counts.index, y=value_counts.values, palette="viridis", ax=ax1)
    ax1.set_xlabel(column, fontsize=12)
    ax1.set_ylabel('Count', fontsize=12)
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right', fontsize=10)

    # Add data labels above each bar
    for index, value in enumerate(value_counts):
        ax1.text(index, value, str(value), ha='center', va='bottom', fontsize=10)

    # Pie plot on the second subplot
    ax2.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', colors=sns.color_palette("viridis", len(value_counts)))
    ax2.axis('equal')

    # Main title for the figure
    fig.suptitle(f'Comparison of {column} Distribution in Train Dataset', fontsize=18)
    
    # Adjust layout and display the figure
    plt.tight_layout()
    plt.show()
    
def extract_coordinates(location_str): # extracts lat and lon from the location feature
    location_str = str(location_str)
    match = re.search(r'\(([^)]+)\)', location_str)
    if match:
        lon, lat = map(float, match.group(1).split(',')) 
        return lat, lon  
    return None

## **Data Preprocessing**

In [ ]:
# Load dataset
df = pd.read_csv('Filming_Permits.csv')

In [ ]:
df.head(5)

In [ ]:
# Continuous and Categorical Variables
continuous_vars, categorical_vars = get_variable_types(df)

print("Continuous Variables:", continuous_vars)
print("Categorical Variables:", categorical_vars)

In [ ]:
# Values of each categorical column
for column in categorical_vars:
    print("column : ",column,"\nunique values : ",df[column].unique(),"\n")

In [ ]:
# Eliminate some features that are not relevant for the analysis
df = df.drop(columns=['APPLICATIONNUMBER','APPLICATIONTYPE','APPLICATIONDESCRIPTION','APPLICATIONFINALIZEDDATE','APPLICATIONEXPIREDATE','STREETNAME','WORKTYPE','WORKTYPEDESCRIPTION','PRIMARYCONTACTLAST','PRIMARYCONTACTFIRST','PRIMARYCONTACTSTREET','PRIMARYCONTACTSTREET2','PRIMARYCONTACTZIP','EMERGENCYCONTACTNAME','LASTINSPECTIONTYPE','LASTINSPTYPEDESCR','LASTINSPECTIONDATE','LASTINSPECTIONRESULT','DIRECTION','SUFFIX','DETAIL'])

In [ ]:
# Convert some features to Date type
df['APPLICATIONISSUEDDATE'] = pd.to_datetime(df['APPLICATIONISSUEDDATE'])
df['APPLICATIONPROCESSEDDATE'] = pd.to_datetime(df['APPLICATIONPROCESSEDDATE'])
df['APPLICATIONENDDATE'] = pd.to_datetime(df['APPLICATIONENDDATE'])
df['APPLICATIONSTARTDATE'] = pd.to_datetime(df['APPLICATIONSTARTDATE'])

In [ ]:
# Get Year, Month of the Start Date
df['year'] = df['APPLICATIONSTARTDATE'].dt.year
df['month'] = df['APPLICATIONSTARTDATE'].dt.month
df['Month-Year'] = df['APPLICATIONSTARTDATE'].dt.to_period('M')

In [ ]:
# Remove any stripping spaces and converting to upper case
df['APPLICATIONNAME'] = df['APPLICATIONNAME'].str.strip().str.upper()

In [ ]:
# Calculate the difference in Dates
df['processing_time_days'] = (df['APPLICATIONPROCESSEDDATE'] - df['APPLICATIONISSUEDDATE']).dt.days 
df['filming_duration_days'] = (df['APPLICATIONENDDATE'] - df['APPLICATIONSTARTDATE']).dt.days

# Remove any NaN values
df = df[df['processing_time_days'] >= 0]
df = df[df['filming_duration_days'] >= 0]

In [ ]:
# Create a coordinate feature based on the lat and lon of the location feature
df['coordinates'] = df['LOCATION'].apply(extract_coordinates)

## **Data Analysis**
### In this analysis, multiple questions will be addressed, and the answers will be presented in the form of detailed analyses.

### Which years had the highest number of permits issued?

In [ ]:
permits_per_year = df['year'].value_counts().sort_index() # Count the number of permits for each year
monthly_permits = df.groupby('Month-Year').size() 

plt.figure(figsize=(10, 6))
permits_per_year.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Permits Issued Per Year')
plt.xlabel('Year')
plt.ylabel('Number of Permits')
plt.show()

plt.figure(figsize=(12, 6))
monthly_permits.plot(kind='line', marker='*', color='skyblue')
plt.title('Number of Permits Issued by Month-Year')
plt.xlabel('Month-Year')
plt.ylabel('Number of Permits')
plt.show()

#### The analysis shows that the majority of permits were issued between 2013 and 2016, with 2016 having the highest number of permits. There is a significant drop in permit issuance after 2016, and very few permits were issued in the following years, especially from 2017 onward.

### Which months have the highest number of permits issued across all years?

In [ ]:
permits_per_month = df['month'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
permits_per_month.plot(kind='bar', color='lightblue', edgecolor='black')
plt.title('Number of Permits Issued by Month')
plt.xlabel('Month')
plt.ylabel('Number of Permits')
plt.xticks(ticks=range(12), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.show()

#### August sees the highest number of permits issued, while December has the fewest. There is a notable increase in permits during the summer months, particularly from July to September, suggesting that more filming activity occurs during this period.

## What are the show/movie with the permits?

In [ ]:
name_counts = df['APPLICATIONNAME'].value_counts().head(10) # gets the top 10 

plt.figure(figsize=(10, 5))
name_counts.plot(kind='bar')
plt.title('Top 10 Shows/Movies with the most permits')
plt.xlabel('Show/Movie Name')
plt.ylabel('Count')
plt.show()

#### The show with the highest number of permits is Chicago Med, with a significant gap compared to the second-highest show.

## What are the show/movie with the most permits on the year 2024?

In [ ]:
df_2024 = df[df['APPLICATIONSTARTDATE'].dt.year == 2024] ## Get all the permit of the year 2024
name_counts_2024 = df_2024['APPLICATIONNAME'].value_counts().head(10)

plt.figure(figsize=(10, 5))
name_counts_2024.plot(kind='bar')
plt.title('Top 10 Shows/Movies with the most permits in 2024')
plt.xlabel('Show/Movie Name')
plt.ylabel('Count')
plt.show()

#### The show with the highest number of permits in 2024 remains Chicago Med, with a significant gap compared to the second-highest show."

### How many permits are closed and how many are open?

In [ ]:
status_counts = df['APPLICATIONSTATUS'].value_counts()

plt.figure(figsize=(10, 5))
status_counts.plot(kind='bar')
plt.title('Number of Application Status')
plt.xlabel('Application Status')
plt.ylabel('Count')
plt.show()

#### As shown in the analysis, the majority of the permits have been closed.

### What are the permits current milestone?

In [ ]:
milestone_counts = df['CURRENTMILESTONE'].value_counts()

plt.figure(figsize=(10, 5))
milestone_counts.plot(kind='bar')
plt.title('Current Milestone')
plt.xlabel('Milestone')
plt.ylabel('Count')
plt.show()

#### This analysis confirms the findings from the previous analysis, showing that the majority of permits have been marked as 'Completed'

### Do all permits have the same comment, or does each permit have a different comment?

In [ ]:
comments_counts = df['COMMENTS'].value_counts().head(10)

comments_counts = comments_counts[comments_counts.index != '.'] # exclude the comment '.'

plt.figure(figsize=(10, 5))
comments_counts.plot(kind='bar')
plt.title('Top 10 Comments')
plt.xlabel('Comments')
plt.ylabel('Count')
plt.show()


#### The most frequent comment associated with the permits refers to the Chicago Fire Department Engine, which may be related to a show or film with a similar theme.

### What are the top cities and states listed as the primary contact locations in the permits?

In [ ]:
cities_counts = df_2024['PRIMARYCONTACTCITY'].value_counts().head(10)
states_counts = df_2024['PRIMARYCONTACTSTATE'].value_counts().head(10)

# Cities
plt.figure(figsize=(10, 5))
cities_counts.plot(kind='bar')
plt.title('Top 10 Cities for Primary Contact in Permits')
plt.xlabel('City')
plt.ylabel('Count')
plt.show()

# States
plt.figure(figsize=(10, 5))
states_counts.plot(kind='bar')
plt.title('Top 5 States for Primary Contact in Permits')
plt.xlabel('State')
plt.ylabel('Count')
plt.show()

#### As we can see, the city and state with the highest number of permits is Chicago, IL

### What type of street closure that has the highest number of permits?

In [ ]:
street_closure_counts = df['STREETCLOSURE'].value_counts().head(10)

plt.figure(figsize=(10, 5))
comments_counts.plot(kind='bar')
plt.title('Top 10 Street Closure Types')
plt.xlabel('Street Closure Types')
plt.ylabel('Count')
plt.show()

### The analysis shows the same result as the comment feature.

### How long does it take to process permits?

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['processing_time_days'], bins=30, color='skyblue', edgecolor='black')
plt.title('Distribution of Permit Processing Times (in Days)')
plt.xlabel('Processing Time (Days)')
plt.ylabel('Number of Permits')
plt.show()

#### The permits are processed on the same day that they are registered

### How many days does each permit allow for filming?

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['filming_duration_days'], bins=30, color='lightgreen', edgecolor='black')
plt.title('Distribution of Filming Durations (in Days)')
plt.xlabel('Filming Duration (Days)')
plt.ylabel('Number of Permits')
plt.show()

#### Most of the permits are for one day. This could be because the show or film typically doesn't shoot at the same location frequently

### Can you show the permits location on a map?

In [ ]:
# Remove NaN values
df_f = df.dropna(subset=['coordinates']) 

# Separate latitude and longitude
df_f['latitude'] = df_f['coordinates'].apply(lambda x: x[0] if x else None)
df_f['longitude'] = df_f['coordinates'].apply(lambda x: x[1] if x else None)

plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.stock_img()
ax.scatter(df_f['latitude'], df_f['longitude'], color='red', marker='o', transform=ccrs.PlateCarree())
plt.title('Permit Locations')
plt.show()

# With zoom in
plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.stock_img()
ax.set_extent([df_f['latitude'].min() - 0.1, df_f['latitude'].max() + 0.1, 
               df_f['longitude'].min() - 0.1, df_f['longitude'].max() + 0.1], crs=ccrs.PlateCarree())
ax.scatter(df_f['latitude'], df_f['longitude'], color='red', marker='o', transform=ccrs.PlateCarree())
plt.title('Permit Locations')
plt.show()